In [14]:
from core.utils import Tibanna
from core import ff_utils
from uuid import UUID
import ast

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

schema_name = {}
profiles = ff_utils.get_metadata('/profiles/', connection=ff, frame = 'raw')
for key, value in profiles.iteritems():
    schema_name[key] = value['id'].split('/')[-1][:-5]

def is_uuid(value):
    #md5 qualifies as uuid
    if '-' not in value:
        return False
    try:
        UUID(value, version=4)
        return True
    except:
        return False

def find_uuids(val):
    vals = []
    if not val:
        return []
    elif isinstance(val, basestring):
        if is_uuid(val):
            vals = [val]
        else:
            return []
    else:
        text =str(val)
        text_list = [i for i in text. split("'") if len(i) == 36]
        vals = [i for i in text_list if is_uuid(i)]
    return vals
 
    
import json
import copy
def record_object(uuid, store, item_uuids):
    #keep list of fields that only exist in frame embedded (revlinks) that you want connected
    
    add_from_embedded = {'file_fastq':['workflow_run_inputs', 'workflow_run_outputs'],
                         'file_processed':['workflow_run_inputs', 'workflow_run_outputs']
                        }
#     add_from_embedded = {}
  
    #find schema name, store as obj_key, create empty list if missing in store

    obj_type = ff_utils.get_metadata(uuid, connection=ff)['@type'][0]

    try:
        obj_key = schema_name[obj_type]
    except:
        #print 'CAN NOT FIND', obj_type, uuid
        return store, item_uuids
    if obj_key not in store:
        store[obj_key] = []

    add_uuids = []
    resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'raw')
    
    if resp['status'] not in ['submission in progress', 'current', 'released', 'released to project', 'in review by lab', 'uploaded']:
        print obj_key, uuid, resp['status']
        
    # add raw frame to store and uuid to list
    if uuid not in item_uuids:
        store[obj_key].append(resp)
        item_uuids.append(uuid)
    # this case should not happen actually
    else:
        return store, item_uuids
    
    fields_to_check = copy.deepcopy(resp)
  
    # check if any field from the embedded frame is required
    add_fields = add_from_embedded.get(obj_key)
    if add_fields:
        emb_resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'embedded')
        for a_field in add_fields:
            field_val = emb_resp.get(a_field)
            if field_val:
                fields_to_check[a_field] = field_val
        
    for key, value in fields_to_check.iteritems():
        # uuid formatted fields to skip
        if key == 'uuid':
            continue
        uuid_in_val = find_uuids(value)
        
        for a_uuid in uuid_in_val:
            if a_uuid not in item_uuids:
                store, item_uuids = record_object(a_uuid, store, item_uuids)   
    return store, item_uuids

In [28]:
release_files = [
"9fff2c05-0f34-435f-b0a4-765bf8a1a5cd",
"0a017a28-c37a-4b71-8f3a-3904f6b47e42",
"b6cd8f8d-27d2-4d3e-b5bb-a5ceb4ee5c3c",
"9135be43-33c6-463d-87c7-9e2b1d56adbd",
"5f605a56-21f8-4c0b-8c29-ef68b31c3e19",
"1f25a58f-08c7-45a3-b3a8-75fbadd92365",
"2156ca45-9773-4bf6-922a-36f573f47136",
"7ec0a86a-79dc-43c0-8e7f-9b10951c4abe",
"5dbcb281-9491-40c2-82d7-dd003c2c61b9",
"090792c3-6f25-4d33-8ca9-7cc259d070bf",
"65e81c71-1785-42ca-a28e-3da4c27f659c",
"7b535bf7-cd95-4a76-8134-3b43a1651b11",
"989e35be-58c8-479d-bb4e-e0cb01705405",
"0b3c36cf-4e44-43b0-a5c9-2c2e56ac8f0f",
"17b4aa3e-5c9c-457e-8413-1a6441f89324",
"32de0079-ec6f-472f-baf5-82af899f23c5",
"188be50a-72a8-4729-a03a-882e86f7739a",
"e3813ca1-f333-4943-8d83-d05829acefb2",
"fdd70fd4-467a-4b69-bf43-b03998980f8e",
"6072a21e-a96b-4d32-ab52-1c099e71ec7c",
"06e24460-5585-4c7f-9b07-3dc67cd2ff07",
"e6e2bd9e-d91d-4296-b227-e3829fa395d6",
"d2f95ea3-1c24-4884-bd68-6d412e0d0eaf",
"2ccadfe8-39da-425d-bc17-aa5a83156d8e",
"9eb40c13-cf85-487c-9819-71ef74a22dcc",
"6093fe05-edf0-4697-80d8-c61c29ef5b7b",
"66681d95-b946-4477-8199-fdac7c58fb65"
]

skip = 3

for i in range(len(release_files)):
    
    if i<skip:
        continue
        
    my_exp_set = release_files[i]
    print i, my_exp_set
    store_dict = {} 
    item_uuids = []
    store, uuids = record_object(my_exp_set, store_dict, item_uuids)
    print(len(uuids))
    print(len(list(set(uuids))))
    
    ## Check which items would be released
    counter = 0
    print 'Experiment Set is', store['experiment_set_replicate'][0]['status'], store['experiment_set_replicate'][0]['uuid']
    if store['experiment_set_replicate'][0]['status'] != 'released':
        print 'set not released'
        break
        
    for a_type in store:
        for raw_data in store[a_type]:
            if raw_data['status'] in ['deleted']: 
                print '    DELETED DATA', a_type ,raw_data['uuid']
                continue        
            if raw_data['status'] not in ['current', 'released', 'released to project']:    
                counter +=1
                print a_type, raw_data['status'], raw_data['uuid']
    print counter
    
    #release = raw_input('Shall I release?')
    release = 'y'
    if release in ['Y', 'y']:
        print "RELEASING ITEMS"
        # release
        counter = 0
        for a_type in store:
            rel_type = 0
            for raw_data in store[a_type]:
                if raw_data['status'] in ['deleted']: 
                    continue        
                if raw_data['status'] not in ['current', 'released', 'released to project']:    
                    counter +=1
                    print a_type, raw_data['status'], raw_data['uuid']
                    patch_data = {'status': "released"}
                    ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,connection=ff)
                    rel_type += 1
            if rel_type > 0:
                print(a_type, rel_type)
        print counter   

    #proc_file_con = raw_input('Shall I connect processed files?')
    proc_file_con = 'y'
    if proc_file_con in ['Y', 'y']:
        ## This part will only work with a single experiment set stored in "store"
        ## my_exp_set is the defined experiment set
        Procs = []
        for prf in store['file_processed']:
            if prf.get('source_experiments'):
                Procs.append([prf['accession'],prf['source_experiments']])
        patches={}
        for p_file, exp in Procs:
            if len(exp) == 1:
                if patches.get(exp[0]):
                    patches[exp[0]].append(p_file)
                else:
                    patches[exp[0]] = [p_file,]

            elif len(exp) == 2:
                if patches.get(my_exp_set):
                    patches[my_exp_set].append(p_file)
                else:
                    patches[my_exp_set] = [p_file,]

        for a_key in patches.keys():
                    patch_data = {'processed_files': patches[a_key]}
                    ff_utils.patch_metadata(patch_data, obj_id=a_key ,connection=ff)
        print 'done adding proc files'
    print
    print

3 9135be43-33c6-463d-87c7-9e2b1d56adbd
93
93
Experiment Set is released 9135be43-33c6-463d-87c7-9e2b1d56adbd
0
RELEASING ITEMS
0
done adding proc files


4 5f605a56-21f8-4c0b-8c29-ef68b31c3e19
92
92
Experiment Set is released 5f605a56-21f8-4c0b-8c29-ef68b31c3e19
workflow_run_sbg in review by lab 88227325-d1b7-45e1-ab52-4052db55b181
workflow_run_sbg in review by lab 0f637c9c-51d6-4b72-9142-0130994eb7c7
workflow_run_sbg in review by lab 2da872ca-89e1-4ba6-9fea-34a992aac14d
workflow_run_sbg in review by lab d52d1c86-0b9f-4079-91f1-8bcc411d1078
workflow_run_sbg in review by lab a6902072-e946-4d64-b0ef-9754081743b0
workflow_run_sbg in review by lab 1237b1ab-d553-4d38-84c1-8991b1e15fde
workflow_run_sbg in review by lab 893a29ad-e8bc-4d6a-b462-321259a59f0e
workflow_run_sbg in review by lab 3354abaf-fdcf-4e2d-a574-9072cb48dbee
quality_metric_pairsqc in review by lab 75b161b8-cc37-4f58-9592-8e94eccaa385
quality_metric_pairsqc in review by lab a5fbd4f3-d2d2-45e3-9957-138194d06d32
file_processed 

quality_metric_fastqc in review by lab 71709573-d737-4462-b6de-c7232e74b128
quality_metric_fastqc in review by lab 6bf6b1d6-5bcf-4d71-8315-fd8df01273bc
quality_metric_fastqc in review by lab 31faad23-f17f-4700-aab4-cf214274e8ae
quality_metric_fastqc in review by lab 2fc3e8df-c5f3-4fd0-8fdd-27093fe51bab
quality_metric_fastqc in review by lab 3b21b323-1749-4d32-8974-5f4a8fa40290
quality_metric_fastqc in review by lab ab13dcc9-b21c-41de-8e0f-8d1a8d762c8f
quality_metric_fastqc in review by lab 5f74486b-1d96-4c84-842c-e866f79bb515
quality_metric_fastqc in review by lab 9f0b29c0-fc5b-41a7-8cf7-ee7858989f04
quality_metric_fastqc in review by lab 8687dab5-1b40-4d54-baae-5823096ed02f
quality_metric_fastqc in review by lab 7c84cdc1-586f-40be-a42b-8c8606afb7ec
(u'quality_metric_fastqc', 12)
workflow_run_sbg in review by lab 496a32da-c89a-4e2e-acf1-be6bddad1629
workflow_run_sbg in review by lab 41735a4b-31f7-4766-87d0-20e85e9207a2
workflow_run_sbg in review by lab 4f6b3ed6-8d95-45f1-9f4c-c6cab7dc4

88
88
Experiment Set is released 2156ca45-9773-4bf6-922a-36f573f47136
quality_metric_pairsqc in review by lab 8853c6c3-d3ac-471d-90bf-4e71090362d8
quality_metric_pairsqc in review by lab 0004edd8-56f5-4b9a-9826-0b98a1fccd8b
file_processed uploaded 7f9a895a-1086-48f9-aeca-ae52c1c8d23c
file_processed uploaded e1b285e2-789c-4dcf-9a6a-bbf875bcff2f
file_processed uploaded ce95310b-69b0-4503-9ba8-bf85073410b6
file_processed uploaded b7c4e342-a855-4636-bfd7-59a1fc08c024
file_processed uploaded 311df0e9-29e0-41a7-9261-f354556aed7c
file_processed uploaded ccf37245-4f7c-4d96-a476-e9ced92a9d12
file_processed uploaded 1fce5277-bbdc-4cd6-854d-9b7ed501f355
file_processed uploaded 4c6680f3-9cf6-452f-9dc5-ecebf376775d
file_processed uploaded 7e6cb130-ab3e-4dde-8764-f44904828419
file_processed uploaded 03378d0f-2bb8-483c-82a8-8801503acafa
workflow_run_awsem in review by lab 125d8009-5c92-4ffc-b29a-c0f8961d0de5
workflow_run_awsem in review by lab 2a571301-73f6-4bb9-9759-aedcf0371a92
workflow_run_awsem i

workflow_run_sbg in review by lab 51be2ddf-8ef5-4fc4-91b8-f0e3a5f92c30
workflow_run_sbg in review by lab 76917e8a-4beb-48dc-8f86-bf404a4fd7ae
workflow_run_sbg in review by lab 1988f64d-f1d3-4e33-9ae2-203a904c0359
workflow_run_sbg in review by lab ed77b00c-88f1-4b7a-a0ca-58ff292429b9
workflow_run_sbg in review by lab 5f273ea6-4ada-4d73-b501-0abdf54bf7c9
workflow_run_sbg in review by lab 669be91d-35b7-4355-85dd-61cb8096e512
(u'workflow_run_sbg', 8)
quality_metric_pairsqc in review by lab 84aa0613-750e-4dcd-9241-151ce6f26bd1
quality_metric_pairsqc in review by lab b616cb0a-ba25-4111-8a0d-dcc634c6ab89
(u'quality_metric_pairsqc', 2)
file_processed uploaded 32b5861c-f0fb-4072-b1b7-179d0919cb21
file_processed uploaded 00971a7c-3240-49cc-ad66-71ed3d9422aa
file_processed uploaded 9ddf55b3-26db-4029-8eab-634320709b8b
file_processed uploaded 97764c14-8cdf-40a2-8013-ed05f32f4d48
file_processed uploaded 9336ce5c-02aa-462f-83fc-0cbfc652eb74
file_processed uploaded 05b6e4db-a303-4a95-b74c-ecdbae444fe

workflow_run_sbg in review by lab 66ac25d9-1040-4340-aede-dc689aae3858
workflow_run_sbg in review by lab 0f66186e-fc47-41f4-a497-a0e7276beb37
workflow_run_sbg in review by lab 83b9c93b-4970-48fd-bf08-50695e23ce3e
workflow_run_sbg in review by lab f0c53870-b6af-4f2a-bfa7-2e56833a5caa
workflow_run_sbg in review by lab 14203068-ff8b-4e0b-a4e6-4572262b8aa4
workflow_run_sbg in review by lab 6a1fbca2-ab57-449a-a25c-551960de9759
(u'workflow_run_sbg', 8)
quality_metric_pairsqc in review by lab efc38ee8-45fa-4716-8f67-4f9dc9b811fe
quality_metric_pairsqc in review by lab 45142364-8fa1-48cc-8492-824ccbd049e0
(u'quality_metric_pairsqc', 2)
file_processed uploaded 64364a47-42d6-447d-bbce-51f6f249e8f5
file_processed uploaded 749eb3cb-9f18-486d-b887-f25cd7b91abb
file_processed uploaded 456f1b81-4d9f-4244-b23b-6091ee64646e
file_processed uploaded 001584f4-4cb6-4f05-a8fe-30b29d2fe598
file_processed uploaded 364b3b67-d6f2-4013-b71e-610c5039a62f
file_processed uploaded dde28b3e-c138-43b6-a81e-256817a4625

...

(u'workflow_run_sbg', 4)
quality_metric_pairsqc in review by lab 6723199c-7757-4fe2-872d-8d6ff5352ce5
(u'quality_metric_pairsqc', 1)
workflow_run_awsem in review by lab 50fb7c36-25a7-495c-829d-93399f2f0a73
workflow_run_awsem in review by lab 1d007947-24e6-4f08-b049-4637ae704d12
workflow_run_awsem in review by lab 21ae8be5-11e1-4243-ac36-4e2bfd75c1b9
workflow_run_awsem in review by lab 9079e71e-edb6-40bb-afbf-bc594649caa3
(u'workflow_run_awsem', 4)
16
done adding proc files


20 06e24460-5585-4c7f-9b07-3dc67cd2ff07
89
89
Experiment Set is released 06e24460-5585-4c7f-9b07-3dc67cd2ff07
workflow_run_sbg in review by lab ef0f2b8d-1116-4f1c-b2ca-75c1fbd3ed08
workflow_run_sbg in review by lab cf4f64a1-f203-4272-a12a-42152e5e1459
workflow_run_sbg in review by lab cc77554a-7bd1-4cd5-8da6-fe4f3a10f281
workflow_run_sbg in review by lab 6c9b41b4-7fc3-4761-99f6-ea2762b73f94
workflow_run_sbg in review by lab 11993413-690f-4b4c-b00f-50138dfc2360
workflow_run_sbg in review by lab 036fdd7c-cb65-45a8-90

quality_metric_fastqc in review by lab 70bb8201-e683-4e25-94c5-df03c42bcaa7
quality_metric_fastqc in review by lab 5b6e347b-dd2d-4679-ae47-09df213d6fb4
quality_metric_fastqc in review by lab 3c88343e-1645-4e6a-bb9f-fd46ad6edc46
quality_metric_fastqc in review by lab 0955d09c-fc45-49ab-a5b6-76dfebacfe9b
quality_metric_fastqc in review by lab bc39d3c4-aff0-43c4-92f8-f4275fda151a
quality_metric_fastqc in review by lab 2e88bc65-b58d-4bb5-8a41-fc04047ab904
quality_metric_fastqc in review by lab 3b600096-555d-47b1-8785-5cb3f7a311e8
quality_metric_fastqc in review by lab 276b97eb-9f5c-421c-87af-2de1198d750e
quality_metric_fastqc in review by lab f670696b-de1d-43cb-9be3-b189b2358a93
quality_metric_fastqc in review by lab bf41d2cf-a11a-4401-99fb-7b5b9a34403f
(u'quality_metric_fastqc', 12)
workflow_run_sbg in review by lab 0af28277-0f47-4a4a-9082-931ed4e4c1e4
workflow_run_sbg in review by lab b83bc70f-a74a-47b9-8170-b827b3ae9cf0
workflow_run_sbg in review by lab 88475c2d-fed8-4658-970b-8345dcafd

file_processed uploaded 73656f91-aa06-4884-8fc1-762e0380ef7c
file_processed uploaded ce6b4749-2638-48fa-b914-75ab4c169bb8
file_processed uploaded c9e0206a-989e-46f6-beef-97aa1ade0b6e
file_processed uploaded 86c43dd8-9c3e-4aa4-810b-cc55b3beb0b8
file_processed uploaded 90cb4a3b-428f-404e-a3d9-c4d4ca71717b
file_processed uploaded f19a09ba-df30-4756-add1-b541af333e04
file_processed uploaded c377953b-0e61-426b-a905-abab2d5a3002
file_processed uploaded 6446ec65-4604-49f5-afc0-05b6739185fd
file_processed uploaded b9c5b83a-1e65-4f50-af29-bc8439b8697c
file_processed to be uploaded by workflow dda077c4-fba5-48a6-b371-e1ac93fa760f
file_processed to be uploaded by workflow 476ab917-bb4a-4a2a-8481-218f117e69c3
file_processed to be uploaded by workflow db69e0b7-991e-4aed-a63c-2f955c34b386
file_processed to be uploaded by workflow 46a5627d-f142-465a-842e-0b53f195394c
file_processed uploaded 77405ae0-d3f7-4718-a977-1768d583a0b4
file_processed uploaded 69f5737a-0302-4575-88dd-94e1216143ae
file_processe

quality_metric_fastqc in review by lab 660d76cd-ff9b-482b-b6b7-6294cfe4589c
quality_metric_fastqc in review by lab 4f6875c6-e9e8-48a9-9fcc-5093a8bb1631
quality_metric_fastqc in review by lab f4263ca5-fef0-4580-97c4-d59025828cd5
quality_metric_fastqc in review by lab e2c5b27d-4136-4f00-b916-161284f227a5
quality_metric_fastqc in review by lab 4ef94ee2-da1c-4ee6-a3ea-65192f4de2b8
quality_metric_fastqc in review by lab 6513457a-2b98-4916-9a7f-cd4a02aa0260
quality_metric_fastqc in review by lab 60c7bdf1-b1c6-4955-95ce-b258663de3d3
quality_metric_fastqc in review by lab 6137d121-d8c4-43e1-99aa-d55b8af69aba
(u'quality_metric_fastqc', 12)
workflow_run_sbg in review by lab d2c7d3a3-558d-47f8-bcb8-f751c452d714
workflow_run_sbg in review by lab 2d8bde74-c8ac-4455-99bf-7e66e7b540f6
workflow_run_sbg in review by lab 84f93d05-2021-445d-855e-f33db3ae3973
workflow_run_sbg in review by lab af77d8ab-90ff-4b36-98a1-942c76db0a51
workflow_run_sbg in review by lab 99c6c289-08f1-4507-a60e-3eff416a2e44
workfl

done adding proc files


26 66681d95-b946-4477-8199-fdac7c58fb65
sop_map 39373842-12e9-4bbb-8f70-270ec1a859ea deleted
file_processed e0a05909-ab82-4770-8a2a-949bc47db786 to be uploaded by workflow
file_processed 5c984ccc-b584-43bb-b7c9-55f8512db874 to be uploaded by workflow
file_processed 82ebb4fd-e4ec-471a-aa11-c1f92eee19ab to be uploaded by workflow
file_processed ec4d514c-226b-4d02-9f6f-204131b1cd0e to be uploaded by workflow
file_processed 22718242-c76d-486e-81d2-763ccf577911 to be uploaded by workflow
file_processed 8a50aa8d-193f-45dd-8f08-88c10ebc55ff to be uploaded by workflow
file_processed d9d6d6af-f929-4abf-a179-37462849c0ba to be uploaded by workflow
file_processed 548f837c-c6fb-407d-92d5-204dea5e861f to be uploaded by workflow
83
83
Experiment Set is released 66681d95-b946-4477-8199-fdac7c58fb65
quality_metric_pairsqc in review by lab 13aefe33-3c57-4e27-8865-ca40b84129b7
file_processed uploaded 76659bf0-c376-4456-b798-de8c003c7620
file_processed uploaded abbdf9af-0243-44c4